# NeRF Training on Google Colab with Automated Metrics

This notebook trains NeRF on the LEGO dataset and automatically tracks all metrics.

**Expected Runtime**: 4-6 hours on Colab GPU (T4)

**What you'll get**:
- Training time
- Test PSNR & SSIM
- All metrics saved automatically
- Downloadable results

## Step 1: Check GPU

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install imageio imageio-ffmpeg configargparse scikit-image tqdm matplotlib -q

print("✓ Dependencies installed")

## Step 3: Mount Google Drive

**Important**: Upload all your files to Google Drive first:
1. Create a folder: `My Drive/nerf_training/`
2. Upload your downloaded files:
   - `run_nerf_with_metrics.py`
   - `run_nerf_helpers.py`
   - `load_blender.py`
   - `load_llff.py`
   - `load_deepvoxels.py`
   - `load_LINEMOD.py`
   - `lego_config.txt`
   - `analyze_results.py`
3. Upload your dataset folder: `data/` (with the lego dataset inside)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive mounted")

## Step 4: Set Up Working Directory

In [ ]:
import os

# Set the path to your files in Google Drive
# CHANGE THIS to match your Google Drive folder structure
DRIVE_PATH = '/content/drive/MyDrive/nerf_training'

# Change to that directory
os.chdir(DRIVE_PATH)

# Verify files exist
print("Checking files...\n")

required_files = [
    'run_nerf_with_metrics.py',
    'run_nerf_helpers.py',
    'load_blender.py',
    'lego_config.txt'
]

all_good = True
for f in required_files:
    if os.path.exists(f):
        print(f"✓ {f}")
    else:
        print(f"✗ {f} NOT FOUND")
        all_good = False

# Check data
data_path = 'data/nerf_synthetic/lego/transforms_train.json'
if os.path.exists(data_path):
    print(f"✓ Dataset found at {data_path}")
else:
    print(f"✗ Dataset NOT FOUND at {data_path}")
    all_good = False

if all_good:
    print("\n✓ All files found! Ready to train.")
else:
    print("\n✗ Some files missing. Please upload them to Google Drive.")

# Show current directory
print(f"\nCurrent directory: {os.getcwd()}")
print(f"\nFiles in directory:")
!ls -lh

## Step 5: Create Necessary Directories

In [ ]:
# Create configs directory if it doesn't exist
!mkdir -p configs

# Move config file to configs directory if needed
if os.path.exists('lego_config.txt') and not os.path.exists('configs/lego_config.txt'):
    !mv lego_config.txt configs/
    print("✓ Moved lego_config.txt to configs/")

# Create logs directory
!mkdir -p logs

print("✓ Directories created")

## Step 6: Verify Dataset

In [ ]:
import json

# Check dataset structure
print("Dataset structure check:\n")

base_path = 'data/nerf_synthetic/lego'

# Check transforms files
for split in ['train', 'val', 'test']:
    transform_file = f'{base_path}/transforms_{split}.json'
    if os.path.exists(transform_file):
        with open(transform_file, 'r') as f:
            data = json.load(f)
        n_frames = len(data['frames'])
        print(f"✓ {split}: {n_frames} images")
    else:
        print(f"✗ {transform_file} not found")

# Count actual image files
for split in ['train', 'val', 'test']:
    img_dir = f'{base_path}/{split}'
    if os.path.exists(img_dir):
        n_images = len([f for f in os.listdir(img_dir) if f.endswith('.png')])
        print(f"  → {split} images on disk: {n_images}")
    else:
        print(f"  → {img_dir} not found")

## Step 7: Preview Configuration

In [ ]:
# Show the configuration
print("Configuration for training:\n")
!cat configs/lego_config.txt

## Step 8: Start Training

**⚠️ IMPORTANT**: This will take 4-6 hours. Make sure:
1. You're using a GPU runtime (Runtime → Change runtime type → GPU)
2. Your Colab session won't time out (keep the tab open or use Colab Pro)
3. You have enough Google Drive storage for checkpoints (~5GB)

The training will:
- Run for 200,000 iterations
- Save checkpoints every 10,000 iterations
- Evaluate on test set every 25,000 iterations
- Print progress every 100 iterations
- Save all metrics automatically

In [ ]:
# Start training
!python run_nerf_with_metrics.py --config configs/lego_config.txt

## Step 9: Analyze Results

In [ ]:
# Run analysis
!python analyze_results.py \
    --log_dir logs/lego_metrics \
    --save_plot logs/lego_metrics/detailed_analysis.png \
    --export_table logs/lego_metrics/results_table.md

## Step 10: View Results

In [ ]:
import json
import numpy as np
from IPython.display import Image, display, Markdown

# Load summary metrics
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    summary = json.load(f)

for key, value in summary.items():
    if value is not None:
        print(f"{key}: {value}")

print("="*60)

# Display training plot
print("\nTraining Metrics Plot:")
display(Image('logs/lego_metrics/training_metrics.png'))

# Display detailed analysis
print("\nDetailed Analysis:")
display(Image('logs/lego_metrics/detailed_analysis.png'))

# Display markdown table
print("\nResults Table (copy this to your report):")
with open('logs/lego_metrics/results_table.md', 'r') as f:
    table_content = f.read()
display(Markdown(table_content))

## Step 11: Download Results

You can download specific files or zip everything:

In [ ]:
# Zip all results
!cd logs && zip -r lego_metrics_results.zip lego_metrics/

print("✓ Results zipped to: logs/lego_metrics_results.zip")
print("\nYou can find this file in your Google Drive at:")
print(f"{DRIVE_PATH}/logs/lego_metrics_results.zip")

# List key files
print("\nKey result files:")
!ls -lh logs/lego_metrics/*.json
!ls -lh logs/lego_metrics/*.npz
!ls -lh logs/lego_metrics/*.png
!ls -lh logs/lego_metrics/*.md

## Step 12: Quick Metrics Check

In [ ]:
# Quick extraction of key metrics for your report
import json

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    metrics = json.load(f)

print("="*60)
print("KEY METRICS FOR YOUR REPORT")
print("="*60)
print(f"\nTraining Time: {metrics['total_training_time_hours']:.2f} hours")
print(f"Test PSNR: {metrics['best_test_psnr']:.2f} dB")
print(f"Test SSIM: {metrics['avg_test_ssim']:.4f}")
print(f"Avg Step Time: {metrics['avg_step_time']:.4f} seconds")
print("\n" + "="*60)
print("\nCopy these values for comparison with 3DGS!")

## Optional: Render Test Images

In [ ]:
# Render test set images (if you want to visualize quality)
!python run_nerf_with_metrics.py \
    --config configs/lego_config.txt \
    --render_only \
    --render_test

print("\nRendered images saved to: logs/lego_metrics/renderonly_test_*/")

## Troubleshooting

### Out of Memory Error
Edit `configs/lego_config.txt` and reduce:
```
N_rand = 512      # Reduce from 1024
chunk = 4096      # Reduce from 8192
netchunk = 65536  # Reduce from 131072
```

### Session Timeout
- Use Colab Pro for longer sessions
- Or train for fewer iterations (change N_iters in the code to 100000)

### Files Not Found
- Make sure all files are uploaded to Google Drive
- Check the DRIVE_PATH variable in Step 4

### Slow Training
- Make sure you're using GPU runtime
- Check GPU type: T4 is standard, A100 is fastest
- Consider using half_res = True in config for faster training